### Scraping Tweets

In [2]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

In [3]:
#obtaining all tweets from account @elonmusk with the defined date range

query = "(from:elonmusk) until:2020-01-01 since:2010-01-01"
tweets = []
limit = 5000


for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.username, tweet.content])
        
df = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet'])
print(df)

# to save to csv
# df.to_csv('tweets.csv')

                          Date      User  \
0    2019-12-31 21:37:06+00:00  elonmusk   
1    2019-12-31 06:59:34+00:00  elonmusk   
2    2019-12-31 06:57:57+00:00  elonmusk   
3    2019-12-31 02:27:28+00:00  elonmusk   
4    2019-12-30 23:27:10+00:00  elonmusk   
...                        ...       ...   
4995 2018-03-23 16:01:55+00:00  elonmusk   
4996 2018-03-23 15:54:12+00:00  elonmusk   
4997 2018-03-23 15:48:36+00:00  elonmusk   
4998 2018-03-23 15:45:50+00:00  elonmusk   
4999 2018-03-23 15:44:38+00:00  elonmusk   

                                                  Tweet  
0             @engineers_feed @physicsJ It’s a bit slow  
1                     @JohnnaCrider1 It’s not ready yet  
2                            @newscientist Explains 🐈 🎥  
3     @teslaownersSV @rhoehn Thanks all Tesla club m...  
4         Rest in peace Syd Mead. Your art will endure.  
...                                                 ...  
4995  @OfficeOfOMAR Yeah, it’s borderline. FB influe...  
4996  @

In [70]:
df1 = df.reset_index().rename(columns={'index':'Id'})
df1.head()

,Id,Date,User,Tweet
0,0,2019-12-31 21:37:06+00:00,elonmusk,@engineers_feed @physicsJ It’s a bit slow
1,1,2019-12-31 06:59:34+00:00,elonmusk,@JohnnaCrider1 It’s not ready yet
2,2,2019-12-31 06:57:57+00:00,elonmusk,@newscientist Explains 🐈 🎥
3,3,2019-12-31 02:27:28+00:00,elonmusk,@teslaownersSV @rhoehn Thanks all Tesla club m...
4,4,2019-12-30 23:27:10+00:00,elonmusk,Rest in peace Syd Mead. Your art will endure.


In [52]:
example = df['Tweet'][100]
print(example)

@flcnhvy @Sofiaan @Erdayastronaut It’s not that these things can’t be done, they just absorb a lot of software engineering time, which has great opportunity cost. There are very few excellent software engineers &amp; merely increasing team size results in awful bloatware.


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype              
---  ------  --------------  -----              
 0   Date    5000 non-null   datetime64[ns, UTC]
 1   User    5000 non-null   object             
 2   Tweet   5000 non-null   object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 117.3+ KB


### 1. VADER Sentiment Scoring 

In [49]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm

sia = SentimentIntensityAnalyzer()

In [51]:
sia.polarity_scores(df['Tweet'][100])

{'neg': 0.06, 'neu': 0.722, 'pos': 0.218, 'compound': 0.835}

In [60]:
res = {}
for i ,row in tqdm(df1.iterrows(), total=len(df1)):
    text = row['Tweet']
    myid = row['Id']
    res[myid] = sia.polarity_scores(text)

In [73]:
vaders = pd.DataFrame(res).T
vaders = vaders.reset_index().rename(columns={'index': 'Id'})
vaders = vaders.merge(df1, how='left')

In [74]:
vaders.head()

,Id,neg,neu,pos,compound,Date,User,Tweet
0,0,0.000,1.000,0.000,0.0000,2019-12-31 21:37:06+00:00,elonmusk,@engineers_feed @physicsJ It’s a bit slow
1,1,0.345,0.655,0.000,-0.2755,2019-12-31 06:59:34+00:00,elonmusk,@JohnnaCrider1 It’s not ready yet
2,2,0.000,1.000,0.000,0.0000,2019-12-31 06:57:57+00:00,elonmusk,@newscientist Explains 🐈 🎥
3,3,0.000,0.748,0.252,0.6588,2019-12-31 02:27:28+00:00,elonmusk,@teslaownersSV @rhoehn Thanks all Tesla club m...
4,4,0.000,0.696,0.304,0.5423,2019-12-30 23:27:10+00:00,elonmusk,Rest in peace Syd Mead. Your art will endure.
